# Import and preprocess of data file

In [1]:
import pandas as pd

#dataset_path = experiment_config["design_state_data"]["session_info"]["dataset"]
df = pd.read_csv(r'C:/Users/psara/Company/Dentrite/DS_Assignment - internship/Screening Test - DS/iris.csv')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [3]:
# Count the number of data points for each class in the 'species' column
class_counts = df['species'].value_counts()

# Print the class counts
print(class_counts)

Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: species, dtype: int64


In [4]:

# Get the unique entries in the 'species' column
unique_species = df['species'].unique()

# Print the unique species entries
print(unique_species)

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


In [5]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [6]:
# Create a mapping from species to integer values
species_mapping = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}

# Apply the mapping to the 'species' column
df['species'] = df['species'].map(species_mapping)

# Print the first few rows of the DataFrame with converted species column
print(df.head())

   sepal_length  sepal_width  petal_length  petal_width  species
0           5.1          3.5           1.4          0.2        0
1           4.9          3.0           1.4          0.2        0
2           4.7          3.2           1.3          0.2        0
3           4.6          3.1           1.5          0.2        0
4           5.0          3.6           1.4          0.2        0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 6.0 KB


# Imputation of null values

In [8]:
# Count the number of missing values in each column
df.isnull().sum()
# No missing values

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

In [9]:
import json
import pandas as pd
from sklearn.impute import SimpleImputer

def impute_missing_values(data, feature_details):
    impute_method = feature_details["feature_details"]["missing_values"]
    impute_value = feature_details["feature_details"]["impute_value"]

    if impute_method == "Impute":
        impute_with = feature_details["feature_details"]["impute_with"]
        if impute_with == "Average of values":
            imputer = SimpleImputer(strategy="mean")
        elif impute_with == "custom":
            imputer = SimpleImputer(strategy="constant", fill_value=impute_value)
        else:
            return data  # No imputation needed

        data[feature_details["feature_name"]] = imputer.fit_transform(data[[feature_details["feature_name"]]])

    return data

# Load the JSON configuration
json_file_path = r"C:\Users\psara\Company\Dentrite\DS_Assignment - internship\Screening Test - DS\algoparams_from_ui_modified.json"
with open(json_file_path, "r") as json_file:
    experiment_config = json.load(json_file)

# Iterate through each feature in the configuration
for feature_name, feature_details in experiment_config["design_state_data"]["feature_handling"].items():
    if feature_details.get("feature_details", {}).get("missing_values") == "Impute":
        df = impute_missing_values(df, feature_details)

# Print the DataFrame after imputation
print(df)


     sepal_length  sepal_width  petal_length  petal_width  species
0             5.1          3.5           1.4          0.2        0
1             4.9          3.0           1.4          0.2        0
2             4.7          3.2           1.3          0.2        0
3             4.6          3.1           1.5          0.2        0
4             5.0          3.6           1.4          0.2        0
..            ...          ...           ...          ...      ...
145           6.7          3.0           5.2          2.3        2
146           6.3          2.5           5.0          1.9        2
147           6.5          3.0           5.2          2.0        2
148           6.2          3.4           5.4          2.3        2
149           5.9          3.0           5.1          1.8        2

[150 rows x 5 columns]


In [10]:
# Extract all columns except the "species" column
features = [col for col in df.columns if col != "species"]

# Create a new DataFrame containing only the selected features
features_df = df[features]

# Print the new DataFrame
print(features_df)


     sepal_length  sepal_width  petal_length  petal_width
0             5.1          3.5           1.4          0.2
1             4.9          3.0           1.4          0.2
2             4.7          3.2           1.3          0.2
3             4.6          3.1           1.5          0.2
4             5.0          3.6           1.4          0.2
..            ...          ...           ...          ...
145           6.7          3.0           5.2          2.3
146           6.3          2.5           5.0          1.9
147           6.5          3.0           5.2          2.0
148           6.2          3.4           5.4          2.3
149           5.9          3.0           5.1          1.8

[150 rows x 4 columns]


# Feature reduction

In [11]:
import json
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.ensemble import RandomForestRegressor

# Load the JSON configuration
json_file_path = r"C:\Users\psara\Company\Dentrite\DS_Assignment - internship\Screening Test - DS\algoparams_from_ui_modified.json"
with open(json_file_path, "r") as json_file:
    experiment_config = json.load(json_file)

# Display the loaded experiment configuration
#print(json.dumps(experiment_config, indent=4))

# Apply feature reduction
feature_reduction_details = experiment_config["design_state_data"]["feature_reduction"]
reduction_method = feature_reduction_details["feature_reduction_method"]



if reduction_method == "No Reduction":
    df_reduced = features_df
    print("No reduction needed.")  # No reduction needed

    

elif reduction_method == "Correlation with target":
    num_of_features_to_keep = int(feature_reduction_details["Correlation with target"]["num_of_features_to_keep"])
    target_column = experiment_config["design_state_data"]["target"]["target"]
    features = [col for col in df.columns if col != target_column]
        
    corr_matrix = df[features + [target_column]].corr()
    corr_with_target = corr_matrix[target_column].abs().sort_values(ascending=False)
    selected_features = corr_with_target.head(num_of_features_to_keep + 1).index.tolist()
        
    df_reduced = df[selected_features]
    print("Reduced DataFrame Shape:", df_reduced.shape)

elif reduction_method == "Tree-based":
    print("tree")
    num_of_features_to_keep = int(feature_reduction_details["Tree-based"]["num_of_features_to_keep"])
    num_of_trees = int(feature_reduction_details["Tree-based"]["num_of_trees"])
    depth_of_trees = int(feature_reduction_details["Tree-based"]["depth_of_trees"])

    target_column = experiment_config["design_state_data"]["target"]["target"]
    features = [col for col in df.columns if col != target_column]
        
    X = df[features]
    y = df[target_column]
        
    rf = RandomForestRegressor(n_estimators=num_of_trees, max_depth=depth_of_trees)
    rf.fit(X, y)
        
    importance = rf.feature_importances_
    feature_importance = list(zip(features, importance))
    feature_importance.sort(key=lambda x: x[1], reverse=True)
    selected_features = [feat for feat, _ in feature_importance[:num_of_features_to_keep]]
        
    df_reduced = df[selected_features + [target_column]]
    print("Reduced DataFrame Shape:", df_reduced.shape)


elif reduction_method == "PCA":
    num_of_features_to_keep = int(feature_reduction_details["Principal Component Analysis"]["num_of_features_to_keep"])
        
    target_column = experiment_config["design_state_data"]["target"]["target"]
    features = [col for col in df.columns if col != target_column]
        
    X = data[features]
    pca = PCA(n_components=num_of_features_to_keep)
    X_reduced = pca.fit_transform(X)
        
    reduced_feature_names = [f"PC{i+1}" for i in range(num_of_features_to_keep)]
    reduced_data = pd.DataFrame(X_reduced, columns=reduced_feature_names)
        
    #return pd.concat([reduced_data, data[target_column]], axis=1)
    df_reduced = pd.concat([reduced_data, features_df["target"]], axis=1)
    print("Reduced DataFrame Shape:", df_reduced.shape)

    
print(df_reduced)

No reduction needed.
     sepal_length  sepal_width  petal_length  petal_width
0             5.1          3.5           1.4          0.2
1             4.9          3.0           1.4          0.2
2             4.7          3.2           1.3          0.2
3             4.6          3.1           1.5          0.2
4             5.0          3.6           1.4          0.2
..            ...          ...           ...          ...
145           6.7          3.0           5.2          2.3
146           6.3          2.5           5.0          1.9
147           6.5          3.0           5.2          2.0
148           6.2          3.4           5.4          2.3
149           5.9          3.0           5.1          1.8

[150 rows x 4 columns]


# Model building

In [12]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from xgboost import XGBClassifier, XGBRegressor
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
import json

# Load the JSON configuration
json_file_path = r"C:\Users\psara\Company\Dentrite\DS_Assignment - internship\Screening Test - DS\algoparams_from_ui_modified.json"
with open(json_file_path, "r") as json_file:
    config = json.load(json_file)

# Get the algorithms from the JSON
algorithms = config["design_state_data"]["algorithms"]

# Initialize the model objects
model_objects = []

for algo_name, algo_params in algorithms.items():
    prediction_type = algo_params.get("prediction_type")
    
    if prediction_type == "Classification":
        if algo_name == "RandomForestClassifier":
            # ... Create RandomForestClassifier object with specified parameters
            rf_params = {
                "n_estimators": algo_params["max_trees"],
                "max_depth": algo_params["max_depth"],
                "min_samples_leaf": algo_params["min_samples_per_leaf_max_value"],
                "random_state": None
            }
            rf_classifier = RandomForestClassifier(**rf_params)
            model_objects.append(rf_classifier)
        elif algo_name == "GBTClassifier":
            # ... Create GradientBoostingClassifier object with specified parameters
            gb_params = {
                "n_estimators": algo_params["num_of_BoostingStages"],
                "learning_rate": algo_params["learningRate"],
                "max_depth": algo_params["max_depth"],
                "subsample": algo_params["max_subsample"],
                "random_state": None
            }
            gb_classifier = GradientBoostingClassifier(**gb_params)
            model_objects.append(gb_classifier)
        elif algo_name == "LogisticRegression":
            # ... Create LogisticRegression object with specified parameters
            logistic_params = {
                "penalty": "l2",
                "solver": "lbfgs",
                "max_iter": algo_params["max_iter"],  # Provide the integer value directly
                "random_state": None
            }
            logistic_classifier = LogisticRegression(**logistic_params)
            model_objects.append(logistic_classifier)
        elif algo_name == "xg_boost":
            xgb_params = {
            # Fill in the parameters based on the JSON values
            }
            xgb_model = XGBClassifier(**xgb_params) if aXGBClassifierlgo_params["use_gradient_boosted_tree"] else XGBRegressor(**xgb_params)
            model_objects.append(xgb_model)    
        elif algo_name == "DecisionTreeClassifier":
            dt_params = {
                "max_depth": algo_params["max_depth"],
                "min_samples_leaf": algo_params["min_samples_per_leaf"],
                "random_state": None
            }
            dt_classifier = DecisionTreeClassifier(**dt_params)
            model_objects.append(dt_classifier)
        elif algo_name == "SVM":
            svm_params = {
                "C": algo_params["c_value"],
                "kernel": "rbf",
                "gamma": "scale"  # You might need to adjust this parameter
            }
            svm_classifier = SVC(**svm_params)
            model_objects.append(svm_classifier)
        elif algo_name == "KNN":
            knn_params = {
            "n_neighbors": algo_params["k_value"],
            "weights": "distance" if algo_params["distance_weighting"] else "uniform",
            "algorithm": algo_params["neighbour_finding_algorithm"],
            "p": algo_params["p_value"]
            }
            knn_classifier = KNeighborsClassifier(**knn_params)
            model_objects.append(knn_classifier)
        # Add conditions for other classification models
        
    elif prediction_type == "Regression":
        if algo_name == "RandomForestRegressor":
            # ... Create RandomForestRegressor object with specified parameters
            rf_params = {
                "n_estimators": algo_params["max_trees"],
                "max_depth": algo_params["max_depth"],
                "min_samples_leaf": algo_params["min_samples_per_leaf_max_value"],
                "random_state": None
            }
            rf_regressor = RandomForestRegressor(**rf_params)
            model_objects.append(rf_regressor)
        elif algo_name == "GBTRegressor":
            # ... Create GradientBoostingRegressor object with specified parameters
            gb_params = {
                "n_estimators": algo_params["num_of_BoostingStages"],
                "max_depth": algo_params["max_depth"],
                "subsample": algo_params["max_subsample"],
                "random_state": None
            }
            gb_regressor = GradientBoostingRegressor(**gb_params)
            model_objects.append(gb_regressor)
        elif algo_name == "LinearRegression":
            # ... Create LinearRegression object with specified parameters
            linear_params = {
                "fit_intercept": True
            }
            linear_regressor = LinearRegression(**linear_params)
            model_objects.append(linear_regressor)
        elif algo_name == "LassoRegression":
            lasso_params = {
            "alpha": (algo_params["min_regparam"] + algo_params["max_regparam"]) / 2,
            "max_iter": algo_params["max_iter"],
            "random_state": None
            }
            lasso_regressor = Lasso(**lasso_params)
            model_objects.append(lasso_regressor)
        elif algo_name == "RidgeRegression":
            ridge_params = {
            "alpha": (algo_params["min_regparam"] + algo_params["max_regparam"]) / 2,
            "max_iter": algo_params["max_iter"],
            "random_state": None
            }
            ridge_regressor = Ridge(**ridge_params)
            model_objects.append(ridge_regressor)            
        elif algo_name == "ElasticNetRegression":
            elastic_net_params = {
            "alpha": (algo_params["min_regparam"] + algo_params["max_regparam"]) / 2,
            "l1_ratio": (algo_params["min_elasticnet"] + algo_params["max_elasticnet"]) / 2,
            "max_iter": algo_params["max_iter"],
            "random_state": None
            }
            elastic_net_regressor = ElasticNet(**elastic_net_params)
            model_objects.append(elastic_net_regressor)
        elif algo_name == "DecisionTreeRegressor":
            dt_params = {
                "max_depth": algo_params["max_depth"],
                "min_samples_leaf": algo_params["min_samples_per_leaf"],
                "random_state": None
            }
            dt_regressor = DecisionTreeRegressor(**dt_params)
            model_objects.append(dt_regressor)
        elif algo_name == "SGD":
            sgd_params = {
            "loss": "log" if algo_params["use_logistics"] else "modified_huber" if algo_params["use_modified_hubber_loss"] else "hinge",
            "penalty": "l1" if algo_params["use_l1_regularization"] else "l2" if algo_params["use_l2_regularization"] else "elasticnet",
            "alpha": algo_params["alpha_value"],
            "max_iter": algo_params["max_iterations"] if algo_params["max_iterations"] else 1000,
            "tol": algo_params["tolerance"],
            "l1_ratio": 0.15,  # You might want to adjust this
            "random_state": None
            }
            sgd_regressor = SGDRegressor(**sgd_params) if algo_params["is_selected"] else None
            model_objects.append(sgd_regressor)
        elif algo_name == "extra_random_trees":
            extra_trees_params = {
            "n_estimators": algo_params["num_of_trees"],
            "max_depth": algo_params["max_depth"],
            "min_samples_leaf": algo_params["min_samples_per_leaf"],
            "n_jobs": None
            }
            extra_trees_regressor = ExtraTreesRegressor(**extra_trees_params)
            model_objects.append(extra_trees_regressor)
        elif algo_name == "neural_network":
            nn_params = {
            "hidden_layer_sizes": tuple(algo_params["hidden_layer_sizes"]),
            "activation": algo_params["activation"],
            "alpha": algo_params["alpha_value"],
            "max_iter": algo_params["max_iterations"],
            "tol": algo_params["convergence_tolerance"],
            "early_stopping": algo_params["early_stopping"],
            "solver": algo_params["solver"],
            "shuffle": algo_params["shuffle_data"],
            "learning_rate_init": algo_params["initial_learning_rate"],
            "batch_size": "auto" if algo_params["automatic_batching"] else None,
            "beta_1": algo_params["beta_1"],
            "beta_2": algo_params["beta_2"],
            "epsilon": algo_params["epsilon"],
            "power_t": algo_params["power_t"],
            "momentum": algo_params["momentum"],
            "nesterovs_momentum": algo_params["use_nesterov_momentum"]
            }
            nn_regressor = MLPRegressor(**nn_params) if algo_params["is_selected"] else None
            model_objects.append(nn_regressor)
        # Add conditions for other regression models
# Now you can use these model objects for training and prediction
for model in model_objects:
    print(model)
    


# Model prediction with hyperparameter tuning

In [13]:
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import confusion_matrix

# Load JSON from file (replace with your file path)
file_path = r"C:\Users\psara\Company\Dentrite\DS_Assignment - internship\Screening Test - DS\algoparams_from_ui_modified.json"
with open(file_path, "r") as json_file:
    design_state_data = json.load(json_file)

# Load your dataset (replace with loading your dataset)
# For example: df = pd.read_csv("your_dataset.csv")

# Iterate through algorithms
for algo_name, algo_params in design_state_data["design_state_data"]["algorithms"].items():
    if algo_params["is_selected"]:
        if design_state_data["design_state_data"]["target"]["prediction_type"] == "Regression":
            label = design_state_data["design_state_data"]["target"]["target"]
            features = df.drop(label, axis=1)
            target = df[label]
            X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=0)
            
            if "RandomForestRegressor" in algo_name:
                model = RandomForestRegressor()
            elif "GBTRegressor" in algo_name:
                model = GradientBoostingRegressor()
            elif "LinearRegression" in algo_name:
                model = LinearRegression()
            elif "LassoRegression" in algo_name:
                model = Lasso()
            elif "RidgeRegression" in algo_name:
                model = Ridge()
            elif "ElasticNetRegression" in algo_name:
                model = ElasticNet()   
            elif "DecisionTreeRegressor" in algo_name:
                model = DecisionTreeRegressor() 
            elif "SGD" in algo_name:
                model = SGDRegressor()   
            elif "extra_random_trees" in algo_name:
                model = ExtraTreesRegressor()                
            elif "neural_network" in algo_name:
                model = MLPRegressor()        
            else:
                continue
            
            param_grid = {key: [value] for key, value in algo_params.items() if key in model.get_params()}
            
            # Parameters to remove
            params_to_remove = ['feature_sampling_statergy', 'is_selected', 'max_trees', 
                                'min_depth', 'min_samples_per_leaf_min_value', 
                                'min_samples_per_leaf_max_value', 'min_trees', 
                                'model_name', 'parallelism', 'fixed_number','max_depth','activation','epsilon','solver']
            
            for param in params_to_remove:
                param_grid.pop(param, None)
            
            grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1)
            grid_search.fit(X_train, y_train)
            
            y_pred = grid_search.predict(X_test)
            
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            
            print(f"Algorithm: {algo_name}")
            print(f"Mean Squared Error: {mse}")
            print(f"R-squared: {r2}")
            print("Best Parameters:", grid_search.best_params_)
            print("\n")
        
        if design_state_data["design_state_data"]["target"]["prediction_type"] == "Classification":
            features = df.drop("species", axis=1)
            target = df["species"]
            X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=0)
            
            if "RandomForestClassifier" in algo_name:
                model = RandomForestClassifier()
            elif "GBTClassifier" in algo_name:
                model = GradientBoostingClassifier()                
            elif "LogisticRegression" in algo_name:
                model = LogisticRegression(solver='liblinear')      
            elif "xg_boost" in algo_name:
                model = XGBClassifier(eval_metric='mlogloss', use_label_encoder=False, early_stopping_rounds=2)  # Set early_stopping_rounds here
                eval_set = [(X_train, y_train), (X_test, y_test)]
            elif "SVM" in algo_name:
                model = SVC(probability=True)  
            elif "KNN" in algo_name:
                model = KNeighborsClassifier()                
            else:
                continue

            param_grid = {key: [value] for key, value in algo_params.items() if key in model.get_params()}
            # Parameters to remove
            params_to_remove = ['feature_sampling_statergy', 'is_selected', 'max_trees', 
                    'min_depth', 'min_samples_per_leaf_min_value', 
                    'min_samples_per_leaf_max_value', 'min_trees', 
                    'model_name', 'parallelism', 'fixed_number','max_depth','activation','epsilon','solver',
                    'min_child_weight','max_depth_of_tree', 'learningRate', 'l1_regularization', 'l2_regularization', 
                    'gamma','sub_sample','col_sample_by_tree']
            for param in params_to_remove:
                param_grid.pop(param, None)

            grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1)
            if "xg_boost" in algo_name:
                grid_search.fit(X_train, y_train, eval_set=eval_set, verbose=10)
            else:
                grid_search.fit(X_train, y_train)
                
           # Predict the class labels
            pred = grid_search.predict(X_test)

            # Get prediction probabilities
            if hasattr(grid_search.best_estimator_, "predict_proba"):
                pred_prob = grid_search.predict_proba(X_test)
            else:
             # Use decision_function() for SVM since it doesn't support predict_proba()
                pred_prob = grid_search.decision_function(X_test)

            # Calculate AUC
            if "SGD" in algo_name:
            # For SGDClassifier, calculate ROC AUC using decision values
                auc_value = roc_auc_score(y_test, pred_prob)
            else:
            # For other classifiers, calculate ROC AUC using predicted probabilities
                auc_value = roc_auc_score(y_test, pred_prob, multi_class='ovr')
   
            

            # Calculate F1 Score
            f1 = f1_score(y_test, pred, average='micro')
        
            # Calculate threshold-based predicted labels for Logistic Regression
            if "LogisticRegression" in algo_name:
                y_pred_labels = (pred_prob[:, 1] >= 0.5).astype(int)
            else:
                y_pred_labels = (pred_prob >= 0.5).astype(int)

            # Calculate Confusion Matrix
            # Calculate Confusion Matrix
            conf_matrix = confusion_matrix(y_test, pred, labels=np.unique(y_test))


            # Calculate Total Cost
            cost_matrix_gain_true_true = design_state_data["design_state_data"]["metrics"]["cost_matrix_gain_for_true_prediction_true_result"]
            cost_matrix_gain_true_false = design_state_data["design_state_data"]["metrics"]["cost_matrix_gain_for_true_prediction_false_result"]
            cost_matrix_gain_false_true = design_state_data["design_state_data"]["metrics"]["cost_matrix_gain_for_false_prediction_true_result"]
            cost_matrix_gain_false_false = design_state_data["design_state_data"]["metrics"]["cost_matrix_gain_for_false_prediction_false_result"]

            total_cost = (conf_matrix[0, 0] * cost_matrix_gain_true_true +
              conf_matrix[0, 1] * cost_matrix_gain_true_false +
              conf_matrix[1, 0] * cost_matrix_gain_false_true +
              conf_matrix[1, 1] * cost_matrix_gain_false_false)

            # Print Results
            print(f"Algorithm: {algo_name}")
            print(f"AUC: {auc_value}")
            print(f"F1 Score: {f1}")
            print(f"Total Cost: {total_cost}")
            print("\n")


Algorithm: RandomForestRegressor
Mean Squared Error: 0.03584041525415714
R-squared: 0.92534802071619
Best Parameters: {}


Algorithm: GBTRegressor
Mean Squared Error: 0.03084022076873657
R-squared: 0.935762922789551
Best Parameters: {}


Algorithm: LinearRegression
Mean Squared Error: 0.037783086103526056
R-squared: 0.9213016327774921
Best Parameters: {}


Algorithm: RidgeRegression
Mean Squared Error: 0.03853924217385969
R-squared: 0.9197266357553433
Best Parameters: {'max_iter': 50}


Algorithm: LassoRegression
Mean Squared Error: 0.30154854796287334
R-squared: 0.37190471159576466
Best Parameters: {'max_iter': 50}


Algorithm: ElasticNetRegression
Mean Squared Error: 0.1415225342753024
R-squared: 0.7052227988433609
Best Parameters: {'max_iter': 50}


Algorithm: DecisionTreeRegressor
Mean Squared Error: 0.07766666666666668
R-squared: 0.8382281469138373
Best Parameters: {}


Algorithm: SGD
Mean Squared Error: 0.05414047044479643
R-squared: 0.8872308468135879
Best Parameters: {}


Algor